# Introduction

This Notebook is adapted from: [PyTorch: GPUx2 EfficientNet fine-tune](https://www.kaggle.com/code/tossimmar/pytorch-gpux2-efficientnet-fine-tune/) Notebook of [@tossimmar](https://www.kaggle.com/tossimmar).

Main change is that we are using a Kaggle predefined model: EfficientNet


# Imported packages

In [ ]:
import io
import glob
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import f1_score

import tensorflow as tf

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.transforms import Compose, Lambda, ToTensor, Normalize, Resize, RandomCrop, TenCrop, RandomHorizontalFlip

# Set configuration

In [ ]:
cfg = {
     "train_files": '/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/train/*.tfrec',
     "valid_files": '/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/*.tfrec',
     "test_files": '/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/test/*.tfrec',
     'model': 'tf_efficientnet_b0',
     'checkpoint_path': '/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth',
     "device": torch.device('cuda' if torch.cuda.is_available() else 'cpu'), # hardware
     "n_classes": 104,
     "n_epochs": 50,                                                            # number of training epochs
     "batch_size": 20,                                                           # training batch size
     "num_prints": 10,                                                            # number of losses to print per epoch
     "train_size": 12753,                                                        # number of training data samples
     "check_freq": 1,                                                            # save model if epoch is a multiple of this
}
cfg["print_freq"] = cfg["train_size"] // (cfg["batch_size"] * cfg["num_prints"]) + 1   #how often to print out update on training evolution

Let's check the configuration.

In [ ]:
cfg

# Utility functions


Here we define two utility functions:
* for converting tfrecords to dataframe, so that we will use easier with our model  
* to visualize the images

In [ ]:
# Utility functions:
# ------------------

def tfrecords_to_dataframe(fp, test = False):
    '''
    Parse data files into rows of a dataframe.
    
    arguments
    ---------
    fp : str
        Data files pattern.
        
    test : bool
        If true, data files correspond to testing data.
    '''
    def parse(pb, test = False):
        d = {'id': tf.io.FixedLenFeature([], tf.string), 'image': tf.io.FixedLenFeature([], tf.string)}
        if not test:
            d['class'] = tf.io.FixedLenFeature([], tf.int64)
        return tf.io.parse_single_example(pb, d)

    df = {'id': [], 'img': []} 
    if not test:
        df['lab'] = []
    for sample in tf.data.TFRecordDataset(glob.glob(fp)).map(lambda pb: parse(pb, test)):
        df['id'].append(sample['id'].numpy().decode('utf-8'))
        df['img'].append(sample['image'].numpy())
        if not test:
            df['lab'].append(sample['class'].numpy())
    return pd.DataFrame(df)

# ------------------------------------------------------------------------------------------------------------------------

def display_images(dataset, n, cols):
    '''
    Display a grid of labelled images of flowers.
    
    arguments
    ---------
    dataset : Dataset
        Dataset containing the flower images and labels.
        
    n : int
        Number of images to display.
        
    cols : int
        Number of columns in the grid.
    '''
    rows = n // cols if n % cols == 0 else n // cols + 1
    plt.figure(figsize = (2 * cols, 2 * rows))
    for i in range(n):
        plt.subplot(rows, cols, i + 1)
        img, lab = dataset[i]
        plt.imshow(img.permute(1, 2, 0).numpy())
        plt.title(str(lab))
        plt.axis('off')
    plt.show()

# Define the classes for train & valid set


We define now the two  classes for train and valid/test set:
* Trainest - for the training set
* Evalset - for evaluation (validation or test) set   


In [ ]:
class Trainset(Dataset):
    '''
    Representation of the training dataset.
    '''
    def __init__(self, frac = 1):
        '''
        arguments
        ---------
        frac : float
            Fraction of data samples to keep.
            
            For example, if frac = 0.5, then a random sample of 50% 
            of the data is kept and the remaining 50% is discarded.
        '''
        super().__init__()
        self.df = tfrecords_to_dataframe(cfg["train_files"]).sample(frac = frac).reset_index(drop = True)
        self.t1 = Lambda(lambda b: Image.open(io.BytesIO(b)))
        self.t2 = Compose([RandomCrop(300), 
                           RandomHorizontalFlip(), 
                           ToTensor(), 
                           Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
    def __len__(self):
        """
            Returns 
                the length of the dataframe with images
        """
        return self.df.shape[0]
    
    def __getitem__(self, i):
        """
        Returns the current item (image + label) from trainset
        Will resize it to 300 x 641 before
            Args
                i: index of current item to return
            Returns
        """
        transform = Compose([self.t1, Resize(np.random.randint(300, 641)), self.t2])
        sample = self.df.iloc[i]
        return transform(sample['img']), sample['lab']


class Evalset(Dataset):
    '''
    Representation of the evaluation datasets.
    '''
    def __init__(self, frac = 1, test = False):
        '''
        Args
            frac : float Fraction of data samples to keep.
            
            For example, if frac = 0.5, then a random sample of 50% 
            of the data is kept and the remaining 50% is discarded.
            
            test : bool
                If true, this dataset contains the testing data. 
                Otherwise, this dataset contains the validation data. 
        Returns
            none
        '''
        super().__init__()
        files = cfg["valid_files"] if not test else cfg["test_files"]
        self.df = tfrecords_to_dataframe(files, test).sample(frac = frac).reset_index(drop = True)
        self.transforms = [Compose([Lambda(lambda b: Image.open(io.BytesIO(b))), 
                                    Resize(scale), 
                                    TenCrop(300), 
                                    Lambda(lambda xs: torch.stack([ToTensor()(x) for x in xs])), 
                                    Lambda(lambda xs: torch.stack([Normalize([0.485, 0.456, 0.406], 
                                                                             [0.229, 0.224, 0.225])(x) for x in xs]))])
                           for scale in [372, 568]]
        self.test = test
        
    def __len__(self):
        """
        Returns
            the valid dataset length
        """
        return self.df.shape[0]
    
    def __getitem__(self, i):
        """
        Generic evaluation class
        Can be used either for validation set or test set
        Args
            i: current item
        Returns 
            either the image and the label (if class is used for validation set) or
            the image only (if class is used for test set)
        """
        sample = self.df.iloc[i]
        imgs = torch.stack([t(sample['img']) for t in self.transforms])
        return imgs, sample['lab'] if not self.test else sample['id']


# Model class

This is the class used to fine-tune EfficientNetB0

In [ ]:
import timm

class EfficientNetB0(nn.Module):
    '''
    EfficientNet B0 fine-tune.
    '''
    def __init__(self, n_classes, learnable_modules = ('classifier',)):
        '''
        Fine tune for EfficientNetB0
        Args
            n_classes : int - Number of classification categories.
            learnable_modules : tuple - Names of the modules to fine-tune.
        Return
            
        '''
        super().__init__()
        
        model = timm.create_model(cfg['model'], 
                                  checkpoint_path=cfg['checkpoint_path'])
        self.efficientnet_b0 = model
        self.efficientnet_b0.classifier = nn.Linear(self.efficientnet_b0.classifier.in_features, n_classes)
        self.efficientnet_b0.requires_grad_(False)
        modules = dict(self.efficientnet_b0.named_modules())
        for name in learnable_modules:
            modules[name].requires_grad_(True)
        
    def forward(self, x):
        """
        Forward function for the fine-tuned model
        Args
            x: 
        Return
            result
        """
        return F.log_softmax(self.efficientnet_b0(x), dim = 1)

# Prepare train, valid and test data

In [ ]:
# Training, validation, and testing data:
# ---------------------------------------
train_set    = Trainset()
train_loader = DataLoader(train_set, batch_size = cfg["batch_size"], shuffle = True, num_workers = 2)
valid_loader = DataLoader(Evalset(frac = 0.20), batch_size = 1, num_workers = 2)
test_loader  = DataLoader(Evalset(test = True), batch_size = 1, num_workers = 2)

# Display sample images (with labels)

<div class="alert alert-info">Display some training images and labels</div>

In [ ]:
display_images(train_set, n = 40, cols = 8)

# Define the optimizer

<div class="alert alert-success">
In the optimizer below, we define five parameter groups (really two as the first four can be combined). 

The first four parameter groups consist of parameters pre-trained on ImageNet and the last parameter group consists of the final affine layer parameters which are trained from scratch. 

We set the pre-trained parameters' learning rate to 0.0001 and the "from scratch" parameters' learning rate to 0.001 -- that is, pre-trained parameters are fine-tuned using a learning rate which is an order of magnitude less than the learning rate of the from scratch parameters. 

The learning rate scheduler decays the learning rates of all groups to zero over the course of training.</div>

In [ ]:
# Modelling components:
# ---------------------
model = nn.DataParallel(EfficientNetB0(n_classes = cfg["n_classes"], 
                                       learnable_modules = (['classifier'])))
model.to(cfg["device"])

optimizer = torch.optim.Adam(params = [
                                       {'params': model.module.efficientnet_b0.classifier.parameters(), 'lr': 2.e-4}], 
                             lr = 2e-5, 
                             weight_decay = 1e-4)

scheduler = CosineAnnealingLR(optimizer, T_max = cfg["n_epochs"])

loss_fn = F.nll_loss

# Training loop

Define train and validate functions.

In [ ]:
def train(train_losses, epoch):
    print()
    print(f'Epoch {epoch}:')
    print('-' * len(f'Epoch {epoch}:'))
    model.train() 
    for i, data in enumerate(train_loader):
        image, label = data
        image = image.to(cfg["device"])
        label = label.to(cfg["device"])
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = loss_fn(outputs, label)
        optimizer.zero_grad()
        # backpropagation
        loss.backward()
        # optimize the weights
        optimizer.step()
        
        # optionaly, print the loss
        if i % cfg["print_freq"] == 0:
            print('Loss {}: {:.3f}'.format(i, loss.item()))
            train_losses.append(loss.item())


def validate(valid_f1s):
    model.eval()
    valid_true_labels = []
    valid_pred_labels = []
    valid_running_loss = []
    counter = 0
    with torch.no_grad():
        for data in valid_loader:
            counter += 1
            image, labels = data
            # append true labels
            valid_true_labels.append(labels.item())

            image = image.view(-1, 3, 300, 300).to(cfg["device"])
            labels = labels.to(cfg["device"])
            # forward pass
            outputs = model(image)
            # calculate the accuracy
            mean_logp = outputs.mean(dim = 0)
            preds = torch.argmax(mean_logp).item()
            valid_pred_labels.append(preds)

    valid_f1 = f1_score(valid_true_labels, valid_pred_labels, average = 'weighted')
    valid_f1s.append(valid_f1)
    
    print()
    print('Validation F1: {:.2f}%'.format(valid_f1 * 100))
    torch.save(model.state_dict(), f'./epoch{epoch // cfg["check_freq"]}.pth')

In [ ]:
# Training loop:
# --------------
train_losses = []  # store train losses at each epoch       
valid_f1s = []  # store f1-score for validation at each epoch                                                          
for epoch in range(cfg["n_epochs"]):
    train(train_losses, epoch)
    # evaluate model (with check_freq)
    if epoch % cfg["check_freq"] == 0:
        validate(valid_f1s)
    scheduler.step()

# Get the optimum epoch

In [ ]:
optimal_epoch = np.argmax(np.array(valid_f1s)) # highest validation F1 epoch / checkpoint frequency

# Validation

<div class="alert alert-success">
    Plot training loss and validation F1
    </div>

In [ ]:
plt.figure(figsize = (12, 4))
plt.subplot(1, 2, 1)
plt.plot(np.arange(len(train_losses)) / cfg["n_epochs"], train_losses, linewidth = 1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Losses')
plt.subplot(1, 2, 2)
plt.plot(np.arange(len(valid_f1s)) * cfg["check_freq"], valid_f1s, linewidth = 1)
plt.vlines(optimal_epoch * cfg["check_freq"], 0, valid_f1s[optimal_epoch], colors = 'black', linestyles = 'dashed', label = f'Optimal epoch ({optimal_epoch * cfg["check_freq"]})')
plt.xlabel('Epoch')
plt.ylabel('Weighted F1')
plt.ylim(0, 1)
plt.title('Validation')
plt.legend(loc = 'lower left')
plt.savefig('plot.png')
plt.show()

# Use the best model

The model that achievel largest validation F1 is loaded and used for submission preparation.

In [ ]:
# Load the model which achieved the largest validation F1:
# --------------------------------------------------------
model = nn.DataParallel(EfficientNetB0(n_classes = cfg["n_classes"], learnable_modules = ())).to(cfg["device"])
model.load_state_dict(torch.load(f'./epoch{optimal_epoch}.pth'))

# Submission

In [ ]:
# Submission:
# -----------
ids = []
preds = []
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        ids.append(y[0])
        mean_logp = model(x.view(-1, 3, 300, 300).to(cfg["device"])).mean(dim = 0)
        preds.append(torch.argmax(mean_logp).item())
submission = pd.DataFrame({'id': ids, 'label': preds})
submission.to_csv('submission.csv', index = False)
submission.head()

# Final note

This Notebook is 100% crafted by and [@tossimmar](https://www.kaggle.com/tossimmar).  

I forked, performed minor changes and run for (my own) learning purposes.   

I also switched to use Kaggle Model(s), namely EfficientNet b0

All credit for this work should go to [@tossimmar](https://www.kaggle.com/tossimmar).  